In [1]:
import os
import numpy as np
from matplotlib.lines import Line2D
from scipy.stats import pearsonr, gaussian_kde, linregress, ttest_ind, sem, zscore
from sklearn.metrics import r2_score, mean_squared_error
from scipy.stats import norm
from scipy.stats import percentileofscore
from sklearn.utils.validation import check_random_state
from math import factorial
from more_itertools import distinct_permutations
import statsmodels.api as sm

import matplotlib.pyplot as plt

import pandas as pd
from sklearn.model_selection import KFold, train_test_split, StratifiedKFold
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
from scipy import stats

import numpy as np
from scipy.stats import linregress
from scipy.stats import pearsonr
import matplotlib.pyplot as plt

from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import KFold, ParameterGrid, train_test_split
from tqdm.notebook import tqdm

import warnings
import random
#from torch.utils.data import SubsetRandomSampler
from sklearn.utils import resample

from sklearn.preprocessing import MinMaxScaler
from statsmodels.stats.diagnostic import linear_harvey_collier
from scipy.stats import chi2
from sklearn.model_selection import train_test_split
from colorama import Fore, Style, init
from sklearn.metrics import roc_auc_score
from collections import Counter

from statsmodels.stats.diagnostic import linear_harvey_collier
from scipy.stats import chi2
from sklearn.model_selection import train_test_split
from colorama import Fore, Style, init
from sklearn.metrics import roc_auc_score

warnings.filterwarnings("ignore")



In [2]:
from scipy.stats import chi2

def summarize_feature(df, feature, df_out, stat_cols=(10, 50), df_offset=2):

    d = df[df["Feature"] == feature]
    if d.empty:
        return df_out  

    for k in stat_cols:
        col = f"hl_statistic_{k}"
        if col in d.columns:
            df_chi = k - df_offset
            combined_p = 1 - chi2.cdf(d[col].mean(), df_chi)
            df_out.loc[feature, f"hl_pvalue_combined_{k}"] = combined_p

    # --- RR summary ---
    if "RR" in d.columns:
        or_mean = d["RR"].mean()
        or_std = d["RR"].std(ddof=1)
        df_out.loc[feature, "RR"] = or_mean
        df_out.loc[feature, "2.5%"] = or_mean - or_std
        df_out.loc[feature, "97.5%"] = or_mean + or_std

    # --- min/max de los IC originales (tal como lo estabas haciendo) ---
    if "2.5%" in d.columns:
        df_out.loc[feature, "minRR"] = d["2.5%"].min()
    if "97.5%" in d.columns:
        df_out.loc[feature, "maxRR"] = d["97.5%"].max()

    return df_out


features = ["Mono", "One", "Two", "Three", "Total"] 




def hosmer_lemeshow(y_true, y_prob, g):
    hl_df = pd.DataFrame({
        "observed": y_true,
        "predicted_probability": y_prob
    }).dropna()


    hl_df["group"] = pd.qcut(hl_df["predicted_probability"], g, duplicates="drop")

    hl_table = hl_df.groupby("group").apply(
        lambda x: pd.Series({
            "observed": x["observed"].sum(),
            "expected": x["predicted_probability"].sum(),
            "total": len(x)
        })
    )

    hl_table["observed_neg"] = hl_table["total"] - hl_table["observed"]
    hl_table["expected_neg"] = hl_table["total"] - hl_table["expected"]


    hl_statistic = (
        ((hl_table["observed"] - hl_table["expected"])**2) / hl_table["expected"] +
        ((hl_table["observed_neg"] - hl_table["expected_neg"])**2) / hl_table["expected_neg"]
    ).sum()


    dof = hl_table.shape[0] - 2
    p_value = 1 - chi2.cdf(hl_statistic, dof)

    return hl_statistic, p_value



def summarize_feature_by_covar(df, covar, feature, df_out, stat_cols=(10, 50), df_offset=2):
    d = df[(df["Covar"] == covar) & (df["Feature"] == feature)]
    if d.empty:
        return df_out

    idx = (covar, feature)

    for k in stat_cols:
        col = f"hl_statistic_{k}"
        if col in d.columns:
            df_chi = k - df_offset
            df_out.loc[idx, f"hl_pvalue_combined_{k}"] = 1 - chi2.cdf(d[col].mean(), df_chi)

    or_mean = d["RR"].mean()
    or_std  = d["RR"].std(ddof=1)
    df_out.loc[idx, "RR"] = or_mean
    df_out.loc[idx, "2.5%"] = or_mean - or_std
    df_out.loc[idx, "97.5%"] = or_mean + or_std

    df_out.loc[idx, "minRR"] = d["2.5%"].min()
    df_out.loc[idx, "maxRR"] = d["97.5%"].max()

    df_out.loc[idx, "n_iter"] = d.shape[0]

    return df_out


## Load data

In [3]:
data = pd.read_parquet('data/BBAG-long.parquet')

In [4]:
data = data[data.country != 'Slovakia'].reset_index(drop =  True)

In [5]:
Counter(data.country)

Counter({'Austria': 4320,
         'Belgium': 4942,
         'Czechia': 5575,
         'Switzerland': 2634,
         'Germany': 4567,
         'Denmark': 2609,
         'Estonia': 7522,
         'Spain': 4912,
         'France': 3640,
         'Hungary': 3010,
         'Italy': 4541,
         'Netherlands': 2546,
         'Poland': 3612,
         'Portugal': 1914,
         'Sweden': 2792,
         'Slovenia': 5291,
         'Luxembourg': 2104,
         'Greece': 2599,
         'Croatia': 2798,
         'Bulgaria': 1953,
         'Cyprus': 1205,
         'Finland': 1982,
         'Lithuania': 2033,
         'Latvia': 1684,
         'Malta': 1259,
         'Romania': 2083})

In [6]:
vars_ = ['Mono', 'One',	'Two',	'Three', 'Total']


In [7]:
data.columns

Index(['record_id', 'country', 'Sex_1F_2M', 'Education', 'Barthel',
       'Diabetes_1Y_0N', 'Hypertension_1Y_0N', 'Heart_Disease_1Y_0N',
       'Physical_activity_1Y_0N', 'Cognition', 'Well_being_domain',
       'Sleep_problems_1Y_0N', 'Audition_problems', 'Vision_problems', 'Age',
       'Mono', 'One', 'Two', 'Three', 'Total', 'total_exposomes',
       'sociopolitical', 'social_physical', 'GDP', 'GDP_per_capita',
       'gender_equal_l', 'Polution_conc_inv', 'Eq', 'free_parties_l',
       'inclu_suff_est', 'cred_elect_est', 'local_dem_est', 'Migration',
       'number_leng_inst', 'number_stable_inst', 'distance', 'Proficiency',
       'Educ_doct', 'Educ_upper_sec', 'Educ_master', 'Educ_bach',
       'year_bilingues', 'delta_time', 'GAP_corrected', 'GAP_bin',
       'y_pred_corrected'],
      dtype='object')

## Odd ratios removing macrosocials (new vars)

### Without Iteration

In [8]:
results_merge_df_all = data.copy()

covar_list = (
                'GDP_per_capita', 'GDP'
    
)

df_directions_odd_all = []
df_directions_cov_odd_all = []

for covar in covar_list:
    for i in vars_:


        c_results_merge_df_all = results_merge_df_all.dropna(subset= [i, covar] + ['delta_time', 'GAP_bin'])
        c_results_merge_df_all.reset_index(drop = True, inplace = True)
        
        y_ols = c_results_merge_df_all['GAP_bin']
        X_ols = c_results_merge_df_all[[i, covar] + ['delta_time']]

        X_train, X_test, y_train, y_test = train_test_split(
            X_ols, y_ols, test_size=0.2, random_state=42
        )

        scaler = MinMaxScaler((0.05, 0.95))

        X_train_scaled = pd.DataFrame(
            scaler.fit_transform(X_train),
            columns=X_train.columns,
            index=X_train.index
        )
        X_test_scaled = pd.DataFrame(
            scaler.transform(X_test),
            columns=X_test.columns,
            index=X_test.index
        )

        X_train_scaled['intercept'] = 1
        X_test_scaled['intercept'] = 1

        model = sm.GLM(y_train, X_train_scaled, family=sm.families.Binomial(link=sm.families.links.log())).fit(disp = 0)

        params = model.params
        conf = np.exp(model.conf_int())
        conf['RR'] = np.exp(params)
        conf['z'] = model.tvalues
        conf['P>|z|'] = model.pvalues
        conf.columns = ['2.5%', '97.5%', 'RR', 'z', 'P>|z|']

        df_i = conf.loc[i:i].copy()
        df_i['Feature'] = i
        df_i['Covar'] = covar
        df_directions_odd_all.append(df_i)

        df_c = conf.loc[covar:covar].copy()
        df_c['Feature'] = f"{i}({covar})"
        df_c['Exposure'] = i          # opcional pero útil
        df_c['Covar'] = covar
        df_directions_cov_odd_all.append(df_c)

df_directions_odd = pd.concat(df_directions_odd_all, axis=0).reset_index(drop=True)
df_directions_cov_odd = pd.concat(df_directions_cov_odd_all, axis=0).reset_index(drop=True)

#df_directions_odd, df_directions_cov_odd


In [9]:
for covar in covar_list:
    c_dfRR = df_directions_odd[df_directions_odd.Covar == covar]
    c_dfRR.to_excel('Results/long/long_RR_-removing-slovakia_new-covar-' + covar + '.xlsx')

    print(covar)
    display(c_dfRR)

GDP_per_capita


,2.5%,97.5%,RR,z,P>|z|,Feature,Covar
0,1.143040,1.228394,1.184948,9.236931,2.536678e-20,Mono,GDP_per_capita
1,0.794754,0.873684,0.833285,-7.550374,4.340102e-14,One,GDP_per_capita
2,0.824134,0.883851,0.853471,-8.878289,6.789646e-19,Two,GDP_per_capita
3,0.952063,1.069230,1.008947,0.300828,7.635459e-01,Three,GDP_per_capita
4,0.813529,0.874394,0.843413,-9.252551,2.191978e-20,Total,GDP_per_capita


GDP


,2.5%,97.5%,RR,z,P>|z|,Feature,Covar
5,1.383517,1.480799,1.431331,20.686429,4.589680e-95,Mono,GDP
6,0.871365,0.956292,0.912841,-3.843665,1.212105e-04,One,GDP
7,0.748609,0.806691,0.777108,-13.228932,5.973009e-40,Two,GDP
8,0.640056,0.711705,0.674930,-14.523972,8.541345e-48,Three,GDP
9,0.674683,0.722193,0.698034,-20.707942,2.937382e-95,Total,GDP


### 1000-Iteration

In [10]:

results_merge_df_all = data.copy()

covar_list = (
                'GDP_per_capita', 'GDP' 
    
)


n_boosts = 10  
test_size = 500 / results_merge_df_all.shape[0]


df_directions_odd_all = []
df_directions_cov_all = []


results_dict = {}  

for covar in covar_list:

    for boosts in range(n_boosts):

        for i in vars_:

            c_results_merge_df_all = results_merge_df_all.dropna(subset= [i, covar] + ['delta_time', 'GAP_bin'])
            c_results_merge_df_all.reset_index(drop = True, inplace = True)
            
            y_ols = c_results_merge_df_all['GAP_bin']
            X_ols = c_results_merge_df_all[[i, covar] + ['delta_time']]

            X_train, X_test, y_train, y_test = train_test_split(
                X_ols, y_ols,
                test_size=test_size,
                stratify=y_ols,
                random_state=boosts
            )

            scaler = MinMaxScaler((0.05, 0.95))

            X_train_scaled = pd.DataFrame(
                scaler.fit_transform(X_train),
                columns=X_train.columns,
                index=X_train.index
            )

            X_test_scaled = pd.DataFrame(
                scaler.transform(X_test),
                columns=X_test.columns,
                index=X_test.index
            )


            X_train_scaled["intercept"] = 1
            X_test_scaled["intercept"] = 1


            model = sm.GLM(y_train, X_train_scaled, family=sm.families.Binomial(link=sm.families.links.log())).fit(disp = 0)
            y_test_pred = model.predict(X_test_scaled)

            key_y = (covar, i, "y")
            key_p = (covar, i, "ypred")
            if key_y not in results_dict:
                results_dict[key_y] = pd.Series(dtype=float)
                results_dict[key_p] = pd.Series(dtype=float)

            results_dict[key_y] = pd.concat([results_dict[key_y], y_test], axis=0)
            results_dict[key_p] = pd.concat([results_dict[key_p], y_test_pred], axis=0)

            params = model.params
            conf = np.exp(model.conf_int())
            conf["RR"] = np.exp(params)
            conf["z"] = model.tvalues
            conf["P>|z|"] = model.pvalues
            conf.columns = ["2.5%", "97.5%", "RR", "z", "P>|z|"]

            hl10_stat, hl10_p = hosmer_lemeshow(y_test, y_test_pred, g=10)
            hl50_stat, hl50_p = hosmer_lemeshow(y_test, y_test_pred, g=50)
            hl100_stat, hl100_p = hosmer_lemeshow(y_test, y_test_pred, g=100)

            df_i = conf.loc[i:i].copy()
            df_i["Feature"] = i
            df_i["Covar"] = covar
            df_i["boosts"] = boosts

            df_i["hl_statistic_10"] = hl10_stat
            df_i["hl_pvalue_10"] = hl10_p
            df_i["hl_statistic_50"] = hl50_stat
            df_i["hl_pvalue_50"] = hl50_p
            df_i["hl_statistic_100"] = hl100_stat
            df_i["hl_pvalue_100"] = hl100_p

            df_directions_odd_all.append(df_i)

            df_c = conf.loc[covar:covar].copy()
            df_c["Feature"] = i + "(" + covar + ")"
            df_c["Exposure"] = i
            df_c["Covar"] = covar
            df_c["boosts"] = boosts

            df_c["hl_statistic_10"] = hl10_stat
            df_c["hl_pvalue_10"] = hl10_p
            df_c["hl_statistic_50"] = hl50_stat
            df_c["hl_pvalue_50"] = hl50_p
            df_c["hl_statistic_100"] = hl100_stat
            df_c["hl_pvalue_100"] = hl100_p

            df_directions_cov_all.append(df_c)

df_directions_odd = pd.concat(df_directions_odd_all, axis=0).reset_index(drop=True)
df_directions_cov_odd = pd.concat(df_directions_cov_all, axis=0).reset_index(drop=True)



In [11]:


features = list(vars_) 
covars = df_directions_odd["Covar"].dropna().unique()

idx = pd.MultiIndex.from_product([covars, features], names=["Covar", "Feature"])
df_directions_odd_f = pd.DataFrame(index=idx)

for c in covars:
    for f in features:
        df_directions_odd_f = summarize_feature_by_covar(
            df_directions_odd, c, f, df_directions_odd_f,
            stat_cols=(10, 50, 100), 
            df_offset=2
        )

df_directions_odd_f = df_directions_odd_f.reset_index()
df_directions_odd_f;


In [12]:
for covar in covar_list:
    c_dfRR = df_directions_odd_f[df_directions_odd_f.Covar == covar]
    c_dfRR.to_excel('Results/long/long_RR_-removing-slovakia_new-covar-' + covar + '-1000-iter.xlsx')

    print(covar)
    display(c_dfRR)

GDP_per_capita


,Covar,Feature,hl_pvalue_combined_10,hl_pvalue_combined_50,hl_pvalue_combined_100,RR,2.5%,97.5%,minRR,maxRR,n_iter
0,GDP_per_capita,Mono,0.249332,0.661382,0.995874,1.193443,1.192220,1.194666,1.153732,1.234723,10.0
1,GDP_per_capita,One,0.313099,0.643629,0.997964,0.833868,0.832158,0.835579,0.796206,0.872893,10.0
2,GDP_per_capita,Two,0.282177,0.537583,0.996998,0.850425,0.849502,0.851347,0.822942,0.879077,10.0
3,GDP_per_capita,Three,0.396806,0.575315,0.993705,0.993769,0.992006,0.995532,0.939925,1.048600,10.0
4,GDP_per_capita,Total,0.249371,0.628389,0.995701,0.837419,0.836558,0.838279,0.809367,0.866299,10.0


GDP


,Covar,Feature,hl_pvalue_combined_10,hl_pvalue_combined_50,hl_pvalue_combined_100,RR,2.5%,97.5%,minRR,maxRR,n_iter
5,GDP,Mono,0.185087,0.384876,0.994101,1.439989,1.438516,1.441462,1.394771,1.487268,10.0
6,GDP,One,0.281055,0.557125,0.995591,0.907515,0.905754,0.909276,0.867598,0.949035,10.0
7,GDP,Two,0.238535,0.521597,0.994122,0.778140,0.777117,0.779163,0.751081,0.806541,10.0
8,GDP,Three,0.127029,0.431415,0.993626,0.670784,0.669497,0.672072,0.637119,0.705420,10.0
9,GDP,Total,0.187010,0.369624,0.994128,0.693838,0.693126,0.694549,0.671748,0.716364,10.0


## Odd ratios removing macrosocials (new vars-several covars)

### Without Iteration

In [13]:
results_merge_df_all = data.copy()

covar_name_group = ['globals',
                    'phisicals',
                    'sociopoliticals',
                    'economics',
                    'linguistics',
                    ]

covar_list = (['sociopolitical', 'social_physical'], 
              ['gender_equal_l', 'Polution_conc_inv', 'Eq'],
              ['free_parties_l',  'inclu_suff_est', 'cred_elect_est', 'local_dem_est'],
              ['GDP', 'GDP_per_capita'],
              ['number_leng_inst', 'number_stable_inst', 'distance', 'Migration'],
              
)

df_directions_odd_all = []

for covar_n in range(len(covar_list)):
    covar_name = covar_name_group[covar_n]
    covar = covar_list[covar_n]

    #print(covar, covar_name)
    
    for i in vars_:

        c_results_merge_df_all = results_merge_df_all.dropna(subset= [i] + covar + ['delta_time', 'GAP_bin'])
        c_results_merge_df_all.reset_index(drop = True, inplace = True)
        
        y_ols = c_results_merge_df_all['GAP_bin']
        X_ols = c_results_merge_df_all[[i] + covar + ['delta_time']]

        X_train, X_test, y_train, y_test = train_test_split(
            X_ols, y_ols, test_size=0.2, random_state=42
        )

        scaler = MinMaxScaler((0.05, 0.95))

        X_train_scaled = pd.DataFrame(
            scaler.fit_transform(X_train),
            columns=X_train.columns,
            index=X_train.index
        )
        X_test_scaled = pd.DataFrame(
            scaler.transform(X_test),
            columns=X_test.columns,
            index=X_test.index
        )

        X_train_scaled['intercept'] = 1
        X_test_scaled['intercept'] = 1

        model = sm.GLM(y_train, X_train_scaled, family=sm.families.Binomial(link=sm.families.links.log())).fit(disp = 0)

        params = model.params
        conf = np.exp(model.conf_int())
        conf['RR'] = np.exp(params)
        conf['z'] = model.tvalues
        conf['P>|z|'] = model.pvalues
        conf.columns = ['2.5%', '97.5%', 'RR', 'z', 'P>|z|']

        df_i = conf.loc[i:i].copy()
        df_i['Feature'] = i
        df_i['Covar'] = covar_name
        df_directions_odd_all.append(df_i)


df_directions_odd = pd.concat(df_directions_odd_all, axis=0).reset_index(drop=True)

#df_directions_odd, df_directions_cov_odd


In [14]:
for covar_n in range(len(covar_list)):
    covar_name = covar_name_group[covar_n]
    covar = covar_list[covar_n]
    
    c_dfRR = df_directions_odd[df_directions_odd.Covar == covar_name]
    c_dfRR.to_excel('Results/long/long_RR_-removing-slovakia_new-covar-several-covars-' + covar_name + '.xlsx')

    print(covar)
    display(c_dfRR)

['sociopolitical', 'social_physical']


,2.5%,97.5%,RR,z,P>|z|,Feature,Covar
0,1.245740,1.354178,1.298828,12.279513,1.166988e-34,Mono,globals
1,0.835643,0.913572,0.873739,-5.934025,2.955967e-09,One,globals
2,0.899910,0.985739,0.941847,-2.578046,9.936073e-03,Two,globals
3,0.773685,0.861184,0.816263,-7.427586,1.105974e-13,Three,globals
4,0.737679,0.801908,0.769123,-12.325604,6.594723e-35,Total,globals


['gender_equal_l', 'Polution_conc_inv', 'Eq']


,2.5%,97.5%,RR,z,P>|z|,Feature,Covar
5,1.341576,1.447788,1.393671,17.077754,2.173240e-65,Mono,phisicals
6,0.936552,1.026154,0.980330,-0.852294,3.940510e-01,One,phisicals
7,0.803008,0.870939,0.836284,-8.630166,6.126300e-18,Two,phisicals
8,0.636098,0.710555,0.672296,-14.060669,6.625708e-45,Three,phisicals
9,0.690089,0.744773,0.716910,-17.107329,1.308652e-65,Total,phisicals


['free_parties_l', 'inclu_suff_est', 'cred_elect_est', 'local_dem_est']


,2.5%,97.5%,RR,z,P>|z|,Feature,Covar
10,1.279478,1.370988,1.324443,15.944797,3.096464e-57,Mono,sociopoliticals
11,0.883510,0.968417,0.924990,-3.330908,8.656311e-04,One,sociopoliticals
12,0.804630,0.865024,0.834281,-9.813402,9.858722e-23,Two,sociopoliticals
13,0.710924,0.785660,0.747359,-11.420042,3.320730e-30,Three,sociopoliticals
14,0.728766,0.780963,0.754413,-15.969619,2.080518e-57,Total,sociopoliticals


['GDP', 'GDP_per_capita']


,2.5%,97.5%,RR,z,P>|z|,Feature,Covar
15,1.117978,1.207021,1.161647,7.664392,1.796810e-14,Mono,economics
16,0.736393,0.815803,0.775082,-9.752535,1.799185e-22,One,economics
17,0.838729,0.903153,0.870345,-7.355591,1.900843e-13,Two,economics
18,1.084131,1.246464,1.162467,4.229334,2.343844e-05,Three,economics
19,0.827933,0.893985,0.860325,-7.683074,1.553156e-14,Total,economics


['number_leng_inst', 'number_stable_inst', 'distance', 'Migration']


,2.5%,97.5%,RR,z,P>|z|,Feature,Covar
20,1.337920,1.469517,1.402175,14.123501,2.721272e-45,Mono,linguistics
21,0.875044,0.964072,0.918480,-3.440274,5.811257e-04,One,linguistics
22,0.774571,0.844417,0.808740,-9.637966,5.527236e-22,Two,linguistics
23,0.717096,0.814828,0.764402,-8.242601,1.685063e-16,Three,linguistics
24,0.679748,0.746749,0.712461,-14.137126,2.242571e-45,Total,linguistics


### 1000-Iteration

In [15]:
results_merge_df_all = data.copy()

covar_name_group = ['globals',
                    'phisicals',
                    'sociopoliticals',
                    'economics',
                    'linguistics',
                    ]

covar_list = (['sociopolitical', 'social_physical'], 
              ['gender_equal_l', 'Polution_conc_inv', 'Eq'],
              ['free_parties_l',  'inclu_suff_est', 'cred_elect_est', 'local_dem_est'],
              ['GDP', 'GDP_per_capita'],
              ['number_leng_inst', 'number_stable_inst', 'distance', 'Migration'],
              
)

n_boosts = 10  
test_size = 500 / results_merge_df_all.shape[0]

df_directions_odd_all = []

for covar_n in range(len(covar_list)):
    covar_name = covar_name_group[covar_n]
    covar = covar_list[covar_n]

    for boosts in range(n_boosts):

        for i in vars_:

            c_results_merge_df_all = results_merge_df_all.dropna(subset= [i] + covar + ['delta_time', 'GAP_bin'])
            c_results_merge_df_all.reset_index(drop = True, inplace = True)
            
            y_ols = c_results_merge_df_all['GAP_bin']
            X_ols = c_results_merge_df_all[[i] + covar + ['delta_time']]


            X_train, X_test, y_train, y_test = train_test_split(
                X_ols, y_ols,
                test_size=test_size,
                stratify=y_ols,
                random_state=boosts
            )

            scaler = MinMaxScaler((0.05, 0.95))

            X_train_scaled = pd.DataFrame(
                scaler.fit_transform(X_train),
                columns=X_train.columns,
                index=X_train.index
            )

            X_test_scaled = pd.DataFrame(
                scaler.transform(X_test),
                columns=X_test.columns,
                index=X_test.index
            )


            X_train_scaled["intercept"] = 1
            X_test_scaled["intercept"] = 1


            model = sm.GLM(y_train, X_train_scaled, family=sm.families.Binomial(link=sm.families.links.log())).fit(disp = 0)
            y_test_pred = model.predict(X_test_scaled)

            key_y = (covar_name, i, "y")
            key_p = (covar_name, i, "ypred")
            if key_y not in results_dict:
                results_dict[key_y] = pd.Series(dtype=float)
                results_dict[key_p] = pd.Series(dtype=float)

            results_dict[key_y] = pd.concat([results_dict[key_y], y_test], axis=0)
            results_dict[key_p] = pd.concat([results_dict[key_p], y_test_pred], axis=0)

            params = model.params
            conf = np.exp(model.conf_int())
            conf["RR"] = np.exp(params)
            conf["z"] = model.tvalues
            conf["P>|z|"] = model.pvalues
            conf.columns = ["2.5%", "97.5%", "RR", "z", "P>|z|"]

            hl10_stat, hl10_p = hosmer_lemeshow(y_test, y_test_pred, g=10)
            hl50_stat, hl50_p = hosmer_lemeshow(y_test, y_test_pred, g=50)
            hl100_stat, hl100_p = hosmer_lemeshow(y_test, y_test_pred, g=100)

            df_i = conf.loc[i:i].copy()
            df_i["Feature"] = i
            df_i["Covar"] = covar_name
            df_i["boosts"] = boosts

            df_i["hl_statistic_10"] = hl10_stat
            df_i["hl_pvalue_10"] = hl10_p
            df_i["hl_statistic_50"] = hl50_stat
            df_i["hl_pvalue_50"] = hl50_p
            df_i["hl_statistic_100"] = hl100_stat
            df_i["hl_pvalue_100"] = hl100_p

            df_directions_odd_all.append(df_i)


df_directions_odd = pd.concat(df_directions_odd_all, axis=0).reset_index(drop=True)



In [16]:


features = list(vars_) 
covars = df_directions_odd["Covar"].dropna().unique()

idx = pd.MultiIndex.from_product([covars, features], names=["Covar", "Feature"])
df_directions_odd_f = pd.DataFrame(index=idx)

for c in covars:
    for f in features:
        df_directions_odd_f = summarize_feature_by_covar(
            df_directions_odd, c, f, df_directions_odd_f,
            stat_cols=(10, 50, 100), 
            df_offset=2
        )

df_directions_odd_f = df_directions_odd_f.reset_index()
df_directions_odd_f;


In [17]:
for covar_n in range(len(covar_list)):
    covar_name = covar_name_group[covar_n]
    covar = covar_list[covar_n]
    
    c_dfRR = df_directions_odd_f[df_directions_odd_f.Covar == covar_name]
    c_dfRR.to_excel('Results/long/long_RR_-removing-slovakia_new-covar-several-covars-' + covar_name + '-1000-iter.xlsx')

    print(covar)
    display(c_dfRR)

['sociopolitical', 'social_physical']


,Covar,Feature,hl_pvalue_combined_10,hl_pvalue_combined_50,hl_pvalue_combined_100,RR,2.5%,97.5%,minRR,maxRR,n_iter
0,globals,Mono,0.350028,0.381067,0.931578,1.306125,1.303874,1.308377,1.255853,1.361366,10.0
1,globals,One,0.099600,0.280267,0.928841,0.874083,0.872583,0.875583,0.837299,0.911742,10.0
2,globals,Two,0.173656,0.192323,0.907847,0.945249,0.943823,0.946675,0.904568,0.987005,10.0
3,globals,Three,0.208157,0.201341,0.879711,0.807491,0.806027,0.808954,0.767359,0.849640,10.0
4,globals,Total,0.356023,0.350629,0.928752,0.764842,0.763528,0.766156,0.733801,0.795462,10.0


['gender_equal_l', 'Polution_conc_inv', 'Eq']


,Covar,Feature,hl_pvalue_combined_10,hl_pvalue_combined_50,hl_pvalue_combined_100,RR,2.5%,97.5%,minRR,maxRR,n_iter
5,phisicals,Mono,0.289053,0.425209,0.962887,1.391578,1.389677,1.393479,1.341467,1.443291,10.0
6,phisicals,One,0.476059,0.593477,0.986233,0.961133,0.959556,0.962709,0.920260,1.004199,10.0
7,phisicals,Two,0.253609,0.539495,0.980113,0.849348,0.847945,0.850752,0.816424,0.883230,10.0
8,phisicals,Three,0.268431,0.463613,0.963041,0.683545,0.682325,0.684764,0.648809,0.720554,10.0
9,phisicals,Total,0.290718,0.393277,0.944517,0.717991,0.717012,0.718970,0.692249,0.744830,10.0


['free_parties_l', 'inclu_suff_est', 'cred_elect_est', 'local_dem_est']


,Covar,Feature,hl_pvalue_combined_10,hl_pvalue_combined_50,hl_pvalue_combined_100,RR,2.5%,97.5%,minRR,maxRR,n_iter
10,sociopoliticals,Mono,0.163596,0.454128,0.993691,1.336668,1.335246,1.338091,1.294256,1.381398,10.0
11,sociopoliticals,One,0.155710,0.503395,0.995192,0.926370,0.924625,0.928114,0.886151,0.968209,10.0
12,sociopoliticals,Two,0.079666,0.440351,0.994280,0.831170,0.830207,0.832134,0.803343,0.860101,10.0
13,sociopoliticals,Three,0.153811,0.368815,0.993592,0.735879,0.734556,0.737201,0.701217,0.771572,10.0
14,sociopoliticals,Total,0.179116,0.452912,0.993030,0.747508,0.746711,0.748305,0.723266,0.772034,10.0


['GDP', 'GDP_per_capita']


,Covar,Feature,hl_pvalue_combined_10,hl_pvalue_combined_50,hl_pvalue_combined_100,RR,2.5%,97.5%,minRR,maxRR,n_iter
15,economics,Mono,0.209145,0.568017,0.997716,1.160823,1.159485,1.162161,1.119373,1.203877,10.0
16,economics,One,0.204330,0.459903,0.995278,0.766495,0.764649,0.768341,0.728901,0.805331,10.0
17,economics,Two,0.292525,0.564993,0.996873,0.873844,0.872726,0.874963,0.844113,0.905314,10.0
18,economics,Three,0.198321,0.511663,0.991166,1.172198,1.169272,1.175125,1.096644,1.251523,10.0
19,economics,Total,0.209163,0.565727,0.997613,0.860944,0.859949,0.861939,0.830102,0.892879,10.0


['number_leng_inst', 'number_stable_inst', 'distance', 'Migration']


,Covar,Feature,hl_pvalue_combined_10,hl_pvalue_combined_50,hl_pvalue_combined_100,RR,2.5%,97.5%,minRR,maxRR,n_iter
20,linguistics,Mono,0.255917,0.447705,0.995487,1.432701,1.429998,1.435405,1.369374,1.498337,10.0
21,linguistics,One,0.186167,0.437822,0.993069,0.908176,0.906277,0.910075,0.866453,0.951485,10.0
22,linguistics,Two,0.133564,0.475922,0.995275,0.800293,0.798726,0.801860,0.767514,0.834461,10.0
23,linguistics,Three,0.084387,0.487610,0.988494,0.753434,0.750702,0.756166,0.706470,0.801049,10.0
24,linguistics,Total,0.264642,0.468964,0.995523,0.697232,0.695912,0.698552,0.666623,0.729544,10.0
